In [2]:
import numpy as np
import pandas as pd
import zarr
import allel

## Zarr2LDNe
 
       Sanjay C Nagi      06/07/20

I have written a snakemake pipeline which subsets and downsamples the zarr genotype arrays in phase 2, and runs LDNe on the populations within. I now want to run this on all of phase 3. As phase 3 is grouped by sample_set, I want to run LDNe on all locations + years that have greater than 15 samples, though this number is currently arbitrary.

Which groups of samples do we want to run LDNe on? see below...
### Phase 3 samples for LDNe

In [18]:
manifest = pd.read_csv("../../data/phase3/Ag1000g.phase3.manifest.tsv", sep="\t")
manifest[manifest.n_samples >= 15]

,country,location,species,year,n_samples
0,Angola,Luanda,coluzzii,2009,81
2,Burkina Faso,Bana,coluzzii,2012,42
3,Burkina Faso,Bana,coluzzii,2014,47
4,Burkina Faso,Bana,gambiae,2012,22
5,Burkina Faso,Bana,gambiae,2014,15
10,Burkina Faso,Pala,gambiae,2012,48
11,Burkina Faso,Pala,gambiae,2014,16
12,Burkina Faso,Souroukoudinga,coluzzii,2012,29
14,Burkina Faso,Souroukoudinga,gambiae,2012,28
15,Burkina Faso,Souroukoudinga,gambiae,2014,15


The idea here would be to (in snakemake) loop through each sample_set (not shown above), then within each sample set loop through each location and finally year, where there is temporal samples. 

At each point we need to extract the appropriate genotypes from the sample set zarrs. We'll also need to filter SNPs, and then can run the downstream LDNe prep.

In [109]:
manifest = pd.read_csv("../../data/phase3/Ag1000g.phase3.manifest.full.tsv", sep="\t")

In [112]:
manifest[manifest.sample_set == _set].head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii
81,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.150,-4.235,F,AG1000G-BF-A,0.024,0.002,gamb_colu,gambiae
82,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.150,-4.235,F,AG1000G-BF-A,0.038,0.002,gamb_colu,gambiae
83,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,0.982,0.002,gamb_colu,coluzzii
84,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,0.990,0.002,gamb_colu,coluzzii
85,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,0.975,0.002,gamb_colu,coluzzii


### Phase 3

In [279]:
all_sets = manifest.sample_set.unique()

chrom = '3L'
n = 10000

In [ ]:
with open(f'../data/Phase3.LDNe.tsv', 'w') as metafile:
    metafile.write("sample_set\tlocation\tyear\tspecies\tchromosome\n")

for _set in all_sets:

    #subset metadata
    metadata = manifest[manifest.sample_set == _set].reset_index(drop=True)
    
    #open genotypes and apply site filters
    Ag_array = zarr.open_array(f"/home/sanj/ag1000g/data/phase3/snp_genotypes/all/{_set}/{chrom}/calldata/GT/", mode = 'r')
    filters = zarr.open(f"/home/sanj/ag1000g/data/phase3/site_filters/dt_20200416/gamb_colu/{chrom}/variants/filter_pass", mode="r")
    positions = zarr.open_array(f"/home/sanj/ag1000g/data/phase3/snp_genotypes/all/sites/{chrom}/variants/POS/", mode='r')
    positions = positions[:][filters[:]]    
    geno = allel.GenotypeDaskArray(Ag_array)
    geno = geno[filters[:]]

    #filter the gff3 to be coding and regulatory regions
    df = allel.gff3_to_dataframe(f"../../data/reference/gff/An.gambiae-PEST-BASEFEATURES_agamP4.12.gff3.gz")
    coding_reg_df = df[~df.type.isin(['chromosome', 'three_prime_UTR','five_prime_UTR',
                      'mRNA', 'CDS', 'exon'])].drop(columns=['source', 'strand', 'phase', 'score'])
    coding_reg_df = coding_reg_df[coding_reg_df.seqid == chrom]

    #get non-centromeric regions. currently chosen by eye based on ag1000g phase1 paper fig1.
    if chrom == '2L':
        centromere = (positions > 3000000)
    elif chrom == '2R':
        centromere = (positions < 57000000)
    elif chrom == '3L':
        centromere = (positions > 2000000)
    elif chrom == '3R':
        centromere = (positions < 50000000)
    elif chrom == 'X':
        centromere = (positions < 21000000)

    positions = allel.SortedIndex(positions[centromere])
    #get boolean array for positions that are coding - allel.locate_ranges so fast!
    coding = positions.locate_ranges(coding_reg_df.start, coding_reg_df.end, strict=False)

    #compress to get noncoding SNPs and remove centromeric regions of low recombination
    #TODO currently using all non-coding regions, alternate option may be to take SNPs x distance from coding regions 
    geno = geno.compress(centromere, axis=0)
    geno = geno.compress(~coding, axis=0) #we want noncoding regions so '~' to get inverse of boolean
    positions = positions[~coding]
    print(f"Filtering out coding regions {chrom}")


    ### loop through combos 
    for loc in metadata.location.unique():

        nmeta = metadata[metadata.location == loc]

        for yr in nmeta.year.unique():

            nmeta = nmeta[nmeta.year == yr]

            for sp in nmeta.species_gambiae_coluzzii.unique():

                #if there is less than 15 samples than skip
                if (nmeta.species_gambiae_coluzzii == sp).sum() < 15:
                    continue

                #filter to species 
                nmeta = nmeta[nmeta.species_gambiae_coluzzii == sp]
                flt = np.array(nmeta.index)
                print(loc, yr, sp, nmeta.shape)

                #filter to correct loc, year, species individuals
                gn = geno.take(flt, axis=1)

                # MAF 0.05 filter
                ac = gn.count_alleles()
                freqs = ac.to_frequencies().compute()
                ALT1 = freqs[:,1] > 0.05
                ALT2 = freqs[:,2] > 0.05
                ALT3 = freqs[:,3] > 0.05
                maf_flt = np.logical_or(ALT1, ALT2, ALT3)
                gn = gn.compress(maf_flt, axis=0)

                #take random sample of n SNPs 
                snp_sample = np.random.choice(gn.shape[0], n, replace=False)
                snp_sample.sort()
                gnr = gn.take(snp_sample, axis=0)
                gnr = np.array(gnr[:])
                gnr = gnr.astype(str)

                pos = positions[maf_flt]
                pos = pos[snp_sample]
                prefix = f'{chrom}_'
                pos_string = [prefix + p for p in pos.astype(str)]

                gnr[gnr == '-1'] = '00' #convert missing alleles 
                dat = np.empty([gnr.shape[0], gnr.shape[1]])

                #join genotypes in same individual 
                print(f"Converting to .dat {_set} {loc} {yr} {sp} {chrom}")
                for x in range(gnr.shape[0]):
                    for y in range(gnr.shape[1]):
                        dat[x,y] = ''.join(gnr[x,y])

                #convert to .dat format genotypes 
                dat = dat.astype(str)
                dat[dat == '0.0'] = '0101'
                dat[dat == '1.0'] = '0102'
                dat[dat == '2.0'] = '0103'
                dat[dat == '3.0'] = '0104'
                dat[dat == '10.0'] = '0201'
                dat[dat == '11.0'] = '0202'
                dat[dat == '12.0'] = '0203'
                dat[dat == '13.0'] = '0204'
                dat[dat == '20.0'] = '0301'
                dat[dat == '21.0'] = '0302'
                dat[dat == '22.0'] = '0303'
                dat[dat == '23.0'] = '0304'
                dat[dat == '30.0'] = '0401'
                dat[dat == '31.0'] = '0402'
                dat[dat == '32.0'] = '0403'
                dat[dat == '33.0'] = '0404'

                #stack population name and transposed genotypes 
                popnames = np.repeat(f"{_set}{loc}{yr}{sp}{chrom}", gnr.shape[1])
                dat = np.column_stack((popnames, dat.T)) #

                #write out .dat file for LDNe 
                with open(f'../data/dat/{_set}.{loc}.{yr}.{sp}.{chrom}.dat', 'w') as datfile:
                    datfile.write(f'{gnr.shape[1]}\t{gnr.shape[0]}\t4\t2\n')
                    datfile.write("\n".join("".join(map(str, x)) for x in pos_string)) 
                    datfile.write("\n")
                    datfile.write("\n".join("\t".join(map(str, x)) for x in dat)) 
                    
                #write metadata file for samples that are included
                with open('../data/Phase3.LDNe.tsv', 'a') as metafile:
                    metafile.write(f'{_set}\t{loc}\t{yr}\t{sp}\t{chrom}\n')
            
                with open('../data/Phase3.LDNe.list', 'a') as listfile:
                    listfile.write(f'{_set}.{loc}.{yr}.{sp}.{chrom}\n')

Filtering out coding regions 3L
Luanda 2009 coluzzii (81, 15)
Converting to .dat AG1000G-AO Luanda 2009 coluzzii 3L
Filtering out coding regions 3L
Pala 2012 gambiae (48, 15)
Converting to .dat AG1000G-BF-A Pala 2012 gambiae 3L
Bana 2012 coluzzii (42, 15)
Converting to .dat AG1000G-BF-A Bana 2012 coluzzii 3L
Souroukoudinga 2012 coluzzii (29, 15)
Converting to .dat AG1000G-BF-A Souroukoudinga 2012 coluzzii 3L
Filtering out coding regions 3L
Bana 2014 coluzzii (47, 15)
Converting to .dat AG1000G-BF-B Bana 2014 coluzzii 3L
Pala 2014 gambiae (16, 15)
Converting to .dat AG1000G-BF-B Pala 2014 gambiae 3L
Souroukoudinga 2014 gambiae (15, 15)
Converting to .dat AG1000G-BF-B Souroukoudinga 2014 gambiae 3L
Filtering out coding regions 3L
Filtering out coding regions 3L
Gbadolite 2015 gambiae (76, 15)
Converting to .dat AG1000G-CD Gbadolite 2015 gambiae 3L
Filtering out coding regions 3L
Filtering out coding regions 3L
Tiassale 2012 coluzzii (80, 15)
Converting to .dat AG1000G-CI Tiassale 2012 co